# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd #импоритируем библиотеку pandas
data = pd.read_csv('/datasets/data.csv') #читаем файл и записиваем его в переенную
data.head() #часть таблицы 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
data.isna().sum() 

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Вывод**

В основном цель кредита: сыграть свадьбу, образовательные курсы, либо крупная покупка(жилье или авто)
Пропущенные значения только в столбцах 'days_employed — общий трудовой стаж в днях' и 'total_income — ежемесячный доход'
Значенения со знаком минус в столбце  days_employed (общий трудовой стаж) это стаж в днях, а значения без знака минус это стаж в часах.(Вполне возможно что в СССР стаж начинался с 8 лет) 
Отрицатеьное значение стажа скорее всего резуьтат работы программы, в которую вносии даты начала стажа и она возвращаа отрицатеьное значение.
___

# Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('mean'))
#заполняем пропущенные значения: все виды доходов заполняем средними значениями 

In [4]:
print(data.isnull().sum())
data = data.fillna(0) #оставшиеся пропуски заменяем на ноль
print(data.isnull().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


In [5]:
data['children'].value_counts()
data['children'] = data['children'].replace(20,2) #В стлобце с количесвтом детей 20 заменяем на 2
#(предположительно лишний ноль добавили), со знаечением -1 заменяем на 1.
data['children'] = data['children'].replace(-1,1)
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [6]:
data['dob_years'].value_counts()
dob_years_mean = data['dob_years'].mean().astype('int')
print(dob_years_mean)
data['dob_years'] = data['dob_years'].replace(0,dob_years_mean)
#Нулевые значения  в столбце возраст мы заменяем на среднее
#значение возраста в этом стоблце, чтобы не потерять важные данные и картина не искажалась(не медиана потому что в столбце
#нет выдающихся значений, можно обойтись средним возрастом(тем более медиана и стреднее одинаковы - 43)).
data['dob_years'].value_counts()

43


35    617
43    614
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [7]:
data['gender'].value_counts()
data['gender'] = data['gender'].replace('XNA','F') #некорректное значение заменяем на F, тк F чаще всего встречается

In [8]:
data.loc[(data['days_employed'] > 0), 'days_employed'] = data['days_employed'] / 24 #в столбце 'общий трудовой стаж в днях'
#переводим часы в дни
data.loc[(data['days_employed'] < 0), 'days_employed'] = data['days_employed'] * (-1) #убираем минус в значениях столбца

In [9]:
days_employed_median =  data['days_employed'].median().astype('int') 
print(days_employed_median)
data['days_employed'] = data['days_employed'].replace(0,days_employed_median)
#заменяем нулевые значения на медианы, тк есть выдающееся значения
total_income_median =  data['total_income'].median().astype('int') 
print(total_income_median)
data['total_income'] = data['total_income'].replace(0,total_income_median)

data['total_income'] = data['total_income'] / 12 #переводим годовую зарплату в месячную

1808
151931


In [10]:
data.head(13)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,21156.303288,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,9340.001175,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,12157.162691,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,22302.379194,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,13218.006489,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,21313.630452,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,20043.830993,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,11318.661183,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,7988.069369,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,12035.494856,покупка жилья для семьи


**Вывод**

В стлобце с количесвтом детей 20 заменяем на 2(предположительно лишний ноль добавили), со знаечением -1 заменяем на 1.
Нулевые значения  в столбце возраст мы заменяем на среднее значение возраста в этом стоблце, чтобы не потерять важные данные и картина не искажалась(не медиана потому что в столбце нет выдающихся значений, можно обойтись средним возрастом(тем более медиана и стреднее одинаковы - 43)).
Заменяем отрицательные значения в столбцах общий 'трудовой стаж в днях' на положительные с помощью метода .loc
Нулевые значения в столбцах 'общий трудовой стаж в днях' и 'ежемесячный доход' заменяем на медианы.
Предположительно годовую зарплату мы перемели в месячную.

В итоге мы не удаляли строк, чем сохрании информацию.

### Замена типа данных

In [11]:
#data.info()
data['days_employed'] = data['days_employed'].astype('int') #переводим значения в столбцах 'общий трудовой стаж в днях' и 
#'ежемесячный доход' в целочисленные для упрощения
data['total_income'] = data['total_income'].astype('int')
data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Мы имеем 5 столбцов с типом данных строки и 7 столбцов с целочисленными значениями.

### Обработка дубликатов

In [12]:
#data['education'].value_counts()

data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
data['gender'] = data['gender'].str.lower()
data['income_type'] = data['income_type'].str.lower()      #приводим все строковые столбцы к нижнему регистру
data['purpose'] = data['purpose'].str.lower()
###
print(data.duplicated().sum()) #определяем количество дубликатов
data = data.drop_duplicates().reset_index(drop= True) #удаляем дубликаты методом .drop_duplicates()
print(data.duplicated().sum()) #проверяем наличие дубликатов

71
0


**Вывод**

В таблице был 71 дубликат, теперь таковых нет.

### Лемматизация

In [13]:
from pymystem3 import Mystem #импортируем библиотеку лемматизации
m = Mystem()
#data['purpose'].value_counts() #смотрим варианты целей для будущей категоризации
def lem_purpose(row):
    lemmas = m.lemmatize(row["purpose"])
    return lemmas
data['lemmased_purpose'] = data.apply(lem_purpose, axis=1)
#определяем и применяем фунцию лемматизации по строкам в нужном нам столбце "цели кредита"

In [14]:
def purpose_by_lem(lem):
    if 'ремонт'in lem:
        return "ремонт жилья"
    if 'недвижимость'in lem or 'жилье' in lem:
        return "покупка недвижимости"
    if 'свадьба' in lem:
        return "проведение свадьбы"
    if  'образование' in lem:
        return "получение образования"
    if 'автомобиль' in lem:
        return "покупка авто"
data['concrete_purpose'] = data['lemmased_purpose'].apply(purpose_by_lem)
#создаем категаризированный столбец с целью кредита с помощью функции
#ремонт жилья я выделил в отдельную категорию так как него нужно значительно меньше чем на покупку самого жилья
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmased_purpose,concrete_purpose
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,21156,покупка жилья,"[покупка, , жилье, \n]",покупка недвижимости
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,9340,приобретение автомобиля,"[приобретение, , автомобиль, \n]",покупка авто
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,12157,покупка жилья,"[покупка, , жилье, \n]",покупка недвижимости
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,22302,дополнительное образование,"[дополнительный, , образование, \n]",получение образования
4,0,14177,53,среднее,1,гражданский брак,1,f,пенсионер,0,13218,сыграть свадьбу,"[сыграть, , свадьба, \n]",проведение свадьбы
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,21313,покупка жилья,"[покупка, , жилье, \n]",покупка недвижимости
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,20043,операции с жильем,"[операция, , с, , жилье, \n]",покупка недвижимости
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,11318,образование,"[образование, \n]",получение образования
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,7988,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,12035,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",покупка недвижимости


**Вывод**

Мы создали столбец с леммами, к которому тепперь можем обращаться.

### Категоризация данных

In [15]:
def availability_of_children(count):
    if count >0:
        return 'есть дети'
    return 'нет детей'
data['have_children'] = data['children'].apply(availability_of_children)
#создаем категаризированный столбец с двумя категориями "есть дети" и "нет детей" с помощью функции

In [16]:
def the_presence_of_a_partner(name_status):
    if name_status == 'женат / замужем' or name_status =='гражданский брак':
        return 'есть партнер'
    return 'нет партнера'
data['the_presence_of_a_partner'] = data['family_status'].apply(the_presence_of_a_partner)
#создаем категаризированный столбец с семейным положением с двумя категориями "есть партнер" и "нет партнера" с помощью функции
#так как по мне гражданский брак и женат / замужем одно и то же

In [17]:
data['total_income'].mean() #14000
data['total_income'].median() #12600
def solvency(income):
    if income <= 6000:
        return "очень низкий"
    if income <= 12000:
        return "низкий"
    if income <= 35000:
        return "средний"
    if income <= 70000:
        return "высокий"
    return 'очень высокий'
data['incom_level'] = data['total_income'].apply(solvency)
#создаем категаризированный столбец с уровнем дохода с помощью функции
#так как средняя зарплата 14 000 а медианная 12 000 я распледелил критерии категорий зарплат таким образом

In [18]:
final_data = pd.DataFrame(data=data, columns=['have_children', 'the_presence_of_a_partner', 'incom_level', 'concrete_purpose', 'debt'])
#создаем таблицу с нужными столбцами на основе старой для упрощения дальнейшей работы 
final_data.head(10)

,have_children,the_presence_of_a_partner,incom_level,concrete_purpose,debt
0,есть дети,есть партнер,средний,покупка недвижимости,0
1,есть дети,есть партнер,низкий,покупка авто,0
2,нет детей,есть партнер,средний,покупка недвижимости,0
3,есть дети,есть партнер,средний,получение образования,0
4,нет детей,есть партнер,средний,проведение свадьбы,0
5,нет детей,есть партнер,средний,покупка недвижимости,0
6,нет детей,есть партнер,средний,покупка недвижимости,0
7,нет детей,есть партнер,низкий,получение образования,0
8,есть дети,есть партнер,низкий,проведение свадьбы,0
9,нет детей,есть партнер,средний,покупка недвижимости,0


**Вывод**

Нам удалось создать 4 столбца с катагаризированными данными.
И создать таблицу с нужными столбцами.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [19]:
data_child_pivot = final_data.pivot_table(index=['have_children'], values='debt', aggfunc='mean')
data_child_pivot.sort_values(by='debt').reset_index()
#создаем сводную таблицу

,have_children,debt
0,нет детей,0.075438
1,есть дети,0.092082


**Вывод** 

Зависимость есть. Чаще возвращают кредит при отсуствии детей: неплтельщиков всего 7.5%
При наличии же детей возвращают реже: 9.2% неплательщиков

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [20]:
data_partner_pivot = final_data.pivot_table(index=['the_presence_of_a_partner'], values='debt', aggfunc='mean')
data_partner_pivot.sort_values(by='debt').reset_index()
#создаем сводную таблицу

,the_presence_of_a_partner,debt
0,есть партнер,0.079988
1,нет партнера,0.085012


**Вывод**

Зависимость есть. При наличии гражанского брака или замужества долг по кредиту возвращают чаще: примерно 8% неплательщиков.
Но разница не слишком велика: при отсутвии партнера доля невозврата в срок 8.5%

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [21]:
data_incom_pivot = final_data.pivot_table(index=['incom_level'], values='debt', aggfunc='mean')
data_incom_pivot.sort_values(by='debt').reset_index()
#создаем сводную таблицу

,incom_level,debt
0,высокий,0.052764
1,очень низкий,0.068754
2,очень высокий,0.078947
3,средний,0.080130
4,низкий,0.086313


**Вывод**

Зависимость есть. Зависимость дохода и доли возврата ожидаемая в категорях высокий-средний-низкий(чем ниже доход - тем больше доля невозврата в срок). Однако выбиваются категории "очень высокий" и "очень низкий": люди с крайне высоким доходом реже возвращают кредит в срок чем люди с крайне низким доходом (7.8% против 6.8% неплательщиков).

- Как разные цели кредита влияют на его возврат в срок?

In [22]:
data_purpose_pivot = final_data.pivot_table(index=['concrete_purpose'], values='debt', aggfunc='mean')
data_purpose_pivot.sort_values(by='debt').reset_index()
#создаем сводную таблицу

,concrete_purpose,debt
0,ремонт жилья,0.057661
1,покупка недвижимости,0.073207
2,проведение свадьбы,0.080034
3,получение образования,0.092200
4,покупка авто,0.093590


**Вывод**

Больше всего доля невозврата в срок у тех кто покупает авто и получает образование(9.2% и 9.3% соотвецтвенно). 
Свадьба так же "невозвратная" категория, целых 8% неплательщиков.
Ну а ремонт жилья категория где меньше всего невозврата в срок: 5.7%.
Покупка недвижимости с её долей неплательщиков в 7.3% где-то посередине.

## Шаг 4. Общий вывод

**1)** Убедительная просьба заказчикам **разобраться с программой по формированию опыта работы**, чтобы она не возвращала отрицательные значения либо значения на порядки большие.

**2)** Так же встречались **пропуски в столбцах опыт работы и зарплата**. Причем в одинаковых строках. Желательно устранить эту проблему.

**3)** Откуда-то в таблице **встретился 71 дубликат**. Устранение дубликатов *будет хорошим тоном* со стороны заказчика и немного упростит работу над будущими пректами.

**4)** Из таблицы видно что цели кредита каждый клиент писал свими словами(из-за чего **она и та же цень записана 5+ способами**, хотя можно было дать им выбор из ограниченного набора целей. И лемматизацию можно уже не применять.

*Общий вывод из полученых процентов к конце можно сделать такой:*

**Много параметров влияет на процент невозврата кредита.
Из проделанной работы можно сделать вывод что чаще возвращают кредит: люди в браке, без детей, с высоким(но не слишком) доходом и цель кредита которых ремонт жилья или его покупка.**
